# Formação Cientista de Dados - DSA

### Machine Learning

## Projeto com Feedback 8 - Modelagem Preditiva em IoT - Previsão de Uso de Energia

### Leonardo Molero

# Análise Exploratória

In [ ]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn import svm
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

# Faz ajustes para não exibir warnings
warnings.filterwarnings("ignore")

# Parametriza impressão dos gráficos dentro do notebook
%matplotlib inline

# Configura fundo azul com barras brancas para os gráficos
sns.set(color_codes=True)

# Configura a exibição da borda das barras nos gráficos
plt.rcParams["patch.force_edgecolor"] = True 

In [ ]:
## Descrição das variáveis
### date: Data da coleta dos dados pelos sensores (datetime)
### Appliances: Uso de energia (em W)
### lights: Potência de energia de eletrodomesticos na casa (em W)
### TXX: Temperatura em um lugar da casa (em Celsius)
### RH_XX: Umidade em um lugar da casa (em %)
### T_out:Temperatura externa (em Celsius) 
### Press_mm_hg: Pressão externa (em mmHg)
### RH_out: Umidade externa (em %) 
### Windspeed: Velocidade do vento (em m/s)
### Visibility: Visibilidade (em Km)
### Tdewpoint: Temperatura do Ponto de Orvalho (em Celsius)
### rv1: Variável aleatória 01
### rv2: Variável aleatória 02
### WeekStatus: Dia útil ou final de semana (weekend ou weekday)
### Day_of_week: Dia da semana
### NSM: Medida do tempo (em segundos)

In [ ]:
# Carrega o dados de treino e de teste usando a coluna 'date' com index
df = pd.read_csv('dados\projeto8-training.csv', index_col='date')
dft = pd.read_csv('dados\projeto8-testing.csv', index_col='date')

In [ ]:
# Checa o tamanho do dataframe de treino
print(df.shape)

In [ ]:
# Checa o tamanho do dataframe de teste
print(dft.shape)

In [ ]:
# Visualiza os dados treino
df.head(10)

In [ ]:
# Verifica os tipos das colunas
df.dtypes

In [ ]:
# Verifica os tipos de colunas agrupados (devido a quantidade de colunas)
df.dtypes.value_counts()

In [ ]:
# Verifica estatísticas dos dados
df.describe()

In [ ]:
# Verifica a distribuição da variável alvo
df['Appliances'].value_counts()

In [ ]:
# Histograma com a distribuição da variável alvo
fig,ax = plt.subplots(figsize=(12,6))
plt.hist('Appliances',data=df,bins=30)
plt.show()

In [ ]:
# Histograma com a distribuição da variável 'lights'
fig,ax = plt.subplots(figsize=(12,6))
plt.hist('lights',data=df, bins=4)
plt.show()

In [ ]:
# Histograma com a distribuição da variável 'NSM'
fig,ax = plt.subplots(figsize=(12,6))
plt.hist('NSM',data=df, bins=30)
plt.show()

In [ ]:
# Distribuição dos do tipo de dia da semana
df['WeekStatus'].value_counts()

In [ ]:
# Verifica a variável alvo por tipo de dia da semana
fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(x='WeekStatus', y='Appliances',data=df)
ax.xaxis.set_label_text("Tipo de Dia da Semana",fontdict= {'size':14})
ax.yaxis.set_label_text("Appliances",fontdict= {'size':14})
plt.show()

In [ ]:
# Distribuição dos dias da semana
df['Day_of_week'].value_counts()

In [ ]:
# Verifica a variável alvo por dia da semana
fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(x='Day_of_week', y='Appliances',data=df,order=['Sunday','Monday','Tuesday',
                                                           'Wednesday','Thursday','Friday','Saturday'])
ax.xaxis.set_label_text("Dia da Semana",fontdict= {'size':14})
ax.yaxis.set_label_text("Appliances",fontdict= {'size':14})
plt.show()

In [ ]:
# Procura e localiza valores nulos
print(df.isnull().values.any())
fig, ax = plt.subplots(figsize=(14,6))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

# Pré-processamento

In [ ]:
# Verifica a correlação das variáveis numéricas
fig,ax = plt.subplots(figsize=(18,12))
sns.heatmap(df.corr(),annot=True,cmap='viridis')

In [ ]:
# Transforma as varíaveis texto em marcações numéricas
le = LabelEncoder()
le.fit(df['WeekStatus'])
df['WeekStatus'] = le.transform(df['WeekStatus'])
le.fit(df['Day_of_week'])
df['Day_of_week'] = le.transform(df['Day_of_week'])

In [ ]:
# Separa as variáveis preditoras da varíavel alvo
x = df.drop(['Appliances'],axis=1)
y = df['Appliances']

In [ ]:
# Cria um modelo de RandomFlorest para verificar a importância das variáveis preditoras
rf = RandomForestRegressor(n_estimators=100)
rf.fit(x,y)

In [ ]:
# Extraindo a importância do modelo Random Florest
importances = rf.feature_importances_
indices = np.argsort(importances)

In [ ]:
# Obtém os índices dos modelo
ind=[]
for i in indices:
    ind.append(x.columns[i])

In [ ]:
# Plot da Importância dos Atributos
fig,ax = plt.subplots(figsize=(10,7))
plt.barh(range(len(indices)), importances[indices])
plt.xlabel('Importância')
plt.ylabel('Atributos')
plt.xticks(np.arange(0,max(importances[indices]), step=0.02))
plt.yticks(range(len(indices)),ind)
plt.show()

In [ ]:
# Seleciona as variáveis de treino com a importâcia igual ou superior a 0.02 para compor os modelos
col_temp = pd.DataFrame({'coluna':ind,'indice':importances[indices]})
cols = np.array(col_temp.coluna[col_temp['indice']>=0.02])
X = df[cols]
X.head()

In [ ]:
# Seleciona as variáveis de teste com a importâcia igual ou superior a 0.02 para compor os modelos
Xt = dft[cols]
yt = dft['Appliances']
Xt.head()

In [ ]:
# Treina Modelo 01 Regressão Linear Múltipla
modelo_1 = LinearRegression()
modelo_1.fit(X,y)
y_pred = modelo_1.predict(X)
previsao = modelo_1.predict(Xt)
print("R^2  dados treino:", r2_score(y,y_pred))
print("RMSE dados treino:", np.sqrt(mean_squared_error(y,y_pred)))
print('\n')
print("R^2  dados teste:", r2_score(yt,previsao))
print("RMSE dados teste:", np.sqrt(mean_squared_error(yt,previsao)))

In [ ]:
# Treina Modelo 02 com SVM
modelo_2 = svm.SVR()
modelo_2.fit(X,y)
y_pred = modelo_2.predict(X)
previsao = modelo_2.predict(Xt)
print("R^2  dados treino:", r2_score(y,y_pred))
print("RMSE dados treino:", np.sqrt(mean_squared_error(y,y_pred)))
print('\n')
print("R^2  dados teste:", r2_score(yt,previsao))
print("RMSE dados teste:", np.sqrt(mean_squared_error(yt,previsao)))

In [ ]:
# Treina Modelo 03 com XGBoost 
modelo_3 = XGBRegressor(objective='reg:squarederror')
modelo_3.fit(X,y)
y_pred = modelo_3.predict(X)
previsao = modelo_3.predict(Xt)
print("R^2  dados treino:", r2_score(y,y_pred))
print("RMSE dados treino:", np.sqrt(mean_squared_error(y,y_pred)))
print('\n')
print("R^2  dados teste:", r2_score(yt,previsao))
print("RMSE dados teste:", np.sqrt(mean_squared_error(yt,previsao)))

# Modelo Preditivo
### Otimização do modelo com XGBoost que apresentou o melhor RMSE

In [ ]:
# Normaliza as variáveis preditoras para tentar melhorar a acurácia do modelo
scaler = Normalizer().fit(X)
xn = scaler.transform(X)
X = pd.DataFrame(xn,columns=[cols])
X.head()

In [ ]:
scaler = Normalizer().fit(Xt)
xnt = scaler.transform(Xt)
Xt = pd.DataFrame(xnt,columns=[cols])
Xt.head()

In [ ]:
# Treina Modelo 04 com XGBoost e dados normalizados
modelo_4 = XGBRegressor(objective='reg:squarederror', n_jobs=-1)
modelo_4.fit(X,y)
y_pred = modelo_4.predict(X)
previsao = modelo_4.predict(Xt)
print("R^2  dados treino:", r2_score(y,y_pred))
print("MSE  dados treino:", np.sqrt(mean_squared_error(y,y_pred)))
print('\n')
print("R^2  dados teste:", r2_score(yt,previsao))
print("MSE  dados teste:", np.sqrt(mean_squared_error(yt,previsao)))

In [ ]:
# Tenta melhorar o modelo com o RandomizedSearchCV
modelo_5 = XGBRegressor(objective='reg:squarederror', n_jobs=-1) 
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
n_iter_search = 20
random_search = RandomizedSearchCV(modelo_5, param_distributions=params,n_iter=n_iter_search)
random_search.fit(X,y)

In [ ]:
# Cria função para exibir relatório com os três melhores resultados do RandomizedSearchCV
def relatorio(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
# Exibe o relatório com o resultado do RandomizedSearchCV
relatorio(random_search.cv_results_)

In [ ]:
# Faz previsões com o melhor modelo
y_pred = random_search.best_estimator_.predict(X)
previsao = random_search.best_estimator_.predict(Xt)
print("R^2  dados treino:", r2_score(y,y_pred))
print("MSE  dados treino:", np.sqrt(mean_squared_error(y,y_pred)))
print('\n')
print("R^2  dados teste:", r2_score(yt,previsao))
print("MSE  dados teste:", np.sqrt(mean_squared_error(yt,previsao)))

In [ ]:
# Houve piora no modelo mesmo com a aplicação de normalização e refinamento dos paramêtros
# Pode ter sido causado pela alta quantidade de outliers detectados na variável alvo
# Plota os outliers da variável alvo
fig,ax = plt.subplots(1,2,figsize=(12,10))
sns.boxplot(df['Appliances'], orient='v', ax=ax[0])
sns.boxplot(dft['Appliances'], orient='v', ax=ax[1])
plt.show()

In [ ]:
# Calcula o intervalo interquartil para filtrar os outliers dos dados de treino
Q1 = df['Appliances'].quantile(0.25)
Q3 = df['Appliances'].quantile(0.75)
IIQ = Q3 - Q1
filtra_outlier = (df['Appliances'] >= Q1 - 1.5 * IIQ) & (df['Appliances'] <= Q3 + 1.5 * IIQ)
df = df.loc[filtra_outlier]
print(df.shape)

In [ ]:
# Calcula o intervalo interquartil para filtrar os outliers dos dados de teste
Q1 = dft['Appliances'].quantile(0.25)
Q3 = dft['Appliances'].quantile(0.75)
IIQ = Q3 - Q1
filtra_outlier = (dft['Appliances'] >= Q1 - 1.5 * IIQ) & (dft['Appliances'] <= Q3 + 1.5 * IIQ)
dft = dft.loc[filtra_outlier]
print(dft.shape)

In [ ]:
# Separa novamente as variáveis preditoras e varíavel alvo
X = df[cols]
y = df['Appliances']
Xt = dft[cols]
yt = dft['Appliances']

In [ ]:
# Novo teste de paramêtros do modelo XGBoost com o RandomizedSearchCV
modelo_6 = XGBRegressor(objective='reg:squarederror', n_jobs=-1) 
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
n_iter_search = 20
random_search = RandomizedSearchCV(modelo_6, param_distributions=params,n_iter=n_iter_search)
random_search.fit(X,y)

In [ ]:
# Faz previsões com o melhor modelo
y_pred = random_search.best_estimator_.predict(X)
previsao = random_search.best_estimator_.predict(Xt)
print("R^2  dados treino:", r2_score(y,y_pred))
print("RMSE dados treino:", np.sqrt(mean_squared_error(y,y_pred)))
print('\n')
print("R^2  dados teste:", r2_score(yt,previsao))
print("RMSE dados teste:", np.sqrt(mean_squared_error(yt,previsao)))

# Melhor Modelo XGBoost com os outliers da variável alvo removidos